# Data Preparing

In [1]:
import os
import csv
import random
import numpy as np
import pandas as pd

In [2]:
corpus_list = ['cejc','mpdd']
situation_list = ['apology','request','thanksgiving']
sen_type_list = ['res']
src_type = 'original' #'translated'
ver_name = '002_translate_all_res_prefix'
context_len = 2

save_dir = f'outputs/context/{ver_name}/{context_len}/'

In [3]:
def get_data_as_list(path):
    data = []
    with open(path, 'r', encoding='utf-8-sig')as f:
        reader = csv.reader(f)
        for row in reader:
            data.append(row[0])
    return data

def get_df(corpus_list, situation_list, sen_type_list, src_type, context_len, train_type):
    target_text = []
    input_text = []
    prefix = []
    for corpus in corpus_list:
        for situation in situation_list:
            for sen_type in sen_type_list:
                f_path = f'/nfs/nas-7.1/yamashita/LAB/dialogue_data/data/{corpus}/{situation}/{context_len}/rewrited_{sen_type}_{train_type}'
                target_text += get_data_as_list(f_path)
                f_path = f'/nfs/nas-7.1/yamashita/LAB/dialogue_data/data/{corpus}/{situation}/{context_len}/{src_type}_{sen_type}_{train_type}'
                input_text += get_data_as_list(f_path)
                prefix += [f'{corpus} {situation} {sen_type}']*len(get_data_as_list(f_path))
    df = pd.DataFrame([prefix,input_text,target_text], index=['prefix','input_text','target_text']).astype(str).T
    return df

# Finetune

In [4]:
import logging
import pandas as pd
from simpletransformers.t5 import T5Model, T5Args

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

train_type = 'train'    
train_df = get_df(corpus_list, situation_list, sen_type_list, src_type, context_len, train_type)

train_type = 'val'  
eval_df = get_df(corpus_list, situation_list, sen_type_list, src_type, context_len, train_type)

# train_df["prefix"] = ""
# eval_df["prefix"] = ""

display(train_df.iloc[:5])
display(eval_df.iloc[:5])

,prefix,input_text,target_text
0,cejc apology res,query: お願いします。 context: あっ。お父さんすいません。あー。,麻煩你了。
1,cejc apology res,query: はい。 context: あー。すいません。いいですか。,是。
2,cejc apology res,query: はい。 context: すみません。あと角煮。角煮。,對。
3,cejc apology res,query: はい。 context: あとそっちにあるよ。おすすめあるよ。お母さんすいませ...,好的。
4,cejc apology res,query: あっ。はーい。どうぞ。 context: あっ。すいません。いいですか?。はい。,啊，來了來了！


,prefix,input_text,target_text
0,cejc apology res,query: はい。 context: うん。すいません。へー。ねえじゃん。ちょっと頼んで。,是的。
1,cejc apology res,query: じゃあ。 context: 間をごめんなさい。ありがとうございます。,不會。
2,cejc apology res,query: はい。はい。 context: 本気で言ってる?。本気で言ってる?。ごめん。ち...,好啦好啦。
3,cejc apology res,query: きゃ。ごめんなさい。わたしねこれさ不愉快じゃない?。この狭い机にさ椅子がはまら...,沒事啦。不好意思誒，跟你講這些不開心的事。哎呦這個桌子太小了，根本塞不下這張椅子。
4,cejc apology res,query: そう。 context: あー。そっか。ごめん。もう今インフルエンザでうちは大...,還好吧？


In [5]:
model_args = T5Args()

model_args.max_seq_length = 128
model_args.length_penalty = 20
model_args.train_batch_size = 2
model_args.eval_batch_size = 2
model_args.num_train_epochs = 20
model_args.evaluate_during_training = True
model_args.evaluate_during_training_steps = 500
model_args.use_multiprocessing = False
model_args.fp16 = False
model_args.early_stopping_metric = 'eval_loss'
model_args.early_stopping_metric_minimize = True
model_args.early_stopping_patience = 3
model_args.use_early_stopping = True
model_args.save_eval_checkpoints = True
model_args.save_eval_checkpoints = False
model_args.learning_rate = 3e-5
model_args.best_model_dir = save_dir+'best_model/'
model_args.output_dir = save_dir+'ckpt/'
model_args.save_model_every_epoch = True
model_args.save_steps = -1
model_args.no_cache = True
model_args.reprocess_input_data = True
model_args.overwrite_output_dir = True
model_args.preprocess_inputs = False
model_args.num_return_sequences = 1
model_args.wandb_project = ver_name
model = T5Model("mt5", "google/mt5-base", args=model_args)
# model = T5Model("mt5", "google/mt5-base", args=model_args, cuda_device=1)
# Train the model
os.environ['WANDB_CONSOLE'] = 'off'
model.train_model(train_df, eval_data=eval_df)

INFO:simpletransformers.t5.t5_utils: Creating features from dataset file at cache_dir/


/home/yamashita/anaconda3/envs/st/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:3216: FutureWarning: `prepare_seq2seq_batch` is deprecated and will be removed in version 5 of 🤗 Transformers. Use the regular `__call__` method to prepare your inputs and the tokenizer under the `with_target_tokenizer` context manager to prepare your targets. See the documentation of your specific tokenizer for more details
  warnings.warn(
INFO:simpletransformers.t5.t5_model: Training started



Using Adafactor for T5


ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: natsukinateyamashita (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.0 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


/home/yamashita/anaconda3/envs/st/lib/python3.9/site-packages/torch/nn/modules/module.py:795: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/home/yamashita/anaconda3/envs/st/lib/python3.9/site-packages/torch/nn/modules/module.py:760: UserWarning: Using non-full backward hooks on a Module that does not return a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_output. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not return a "
/home/yamashita/anaconda3/envs/st/lib/python3.9/site-packages/transformers/o

...

INFO:simpletransformers.t5.t5_model: No improvement in eval_loss
INFO:simpletransformers.t5.t5_model: Current step: 3
INFO:simpletransformers.t5.t5_model: Early stopping patience: 3
INFO:simpletransformers.t5.t5_utils: Creating features from dataset file at cache_dir/


/home/yamashita/anaconda3/envs/st/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:3216: FutureWarning: `prepare_seq2seq_batch` is deprecated and will be removed in version 5 of 🤗 Transformers. Use the regular `__call__` method to prepare your inputs and the tokenizer under the `with_target_tokenizer` context manager to prepare your targets. See the documentation of your specific tokenizer for more details
  warnings.warn(


INFO:simpletransformers.t5.t5_model: Patience of 3 steps reached
INFO:simpletransformers.t5.t5_model: Training terminated.


INFO:simpletransformers.t5.t5_model: Training of google/mt5-base model complete. Saved to outputs/context/002_translate_all_res_prefix/2/ckpt/.


(12500,
 {'global_step': [500,
   659,
   1000,
   1318,
   1500,
   1977,
   2000,
   2500,
   2636,
   3000,
   3295,
   3500,
   3954,
   4000,
   4500,
   4613,
   5000,
   5272,
   5500,
   5931,
   6000,
   6500,
   6590,
   7000,
   7249,
   7500,
   7908,
   8000,
   8500,
   8567,
   9000,
   9226,
   9500,
   9885,
   10000,
   10500,
   10544,
   11000,
   11203,
   11500,
   11862,
   12000,
   12500],
  'eval_loss': [9.629975359614303,
   8.469074708659475,
   6.178383646941766,
   5.377788869346061,
   5.057911128532596,
   4.392729849350162,
   4.388381894041852,
   4.021890192497067,
   3.9604608213029255,
   3.761639449654556,
   3.6357487556411,
   3.5941992285774975,
   3.4964386559114224,
   3.486486778026674,
   3.4383866365362956,
   3.406430648594368,
   3.371679657843055,
   3.3316610339211254,
   3.305308954017918,
   3.2602866466452434,
   3.2603491719176128,
   3.2128344534373863,
   3.219582423204329,
   3.1929965120990103,
   3.1616232177106345,
   3.176577

# Test

In [9]:

import logging
import sacrebleu
import pandas as pd
from simpletransformers.t5 import T5Model, T5Args


logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)


model_args = T5Args()
model_args.max_length = 128
model_args.length_penalty = 20
model_args.num_beams = 10

model = T5Model("mt5", save_dir+"best_model/", args=model_args)
# model = T5Model("mt5", save_dir+"best_model/", args=model_args, cuda_device=1)

In [10]:
train_type = 'test'  
eval_df = get_df(corpus_list, situation_list, sen_type_list, src_type, context_len, train_type)

to_ja_truth = eval_df.loc[eval_df["prefix"].str.contains("mpdd")]["target_text"].tolist()
to_ja_input = eval_df.loc[eval_df["prefix"].str.contains("mpdd")]["input_text"].tolist()
to_ja_prefix = eval_df.loc[eval_df["prefix"].str.contains("mpdd")]["prefix"].tolist()

to_zh_truth = eval_df.loc[eval_df["prefix"].str.contains("cejc")]["target_text"].tolist()
to_zh_input = eval_df.loc[eval_df["prefix"].str.contains("cejc")]["input_text"].tolist()
to_zh_prefix = eval_df.loc[eval_df["prefix"].str.contains("cejc")]["prefix"].tolist()

# to_ja_input = [": " + input_text for input_text in to_ja_input]
# to_zh_input = [": " + input_text for input_text in to_zh_input]
to_ja_input = [prefix + ": " + input_text for prefix, input_text in zip(to_ja_prefix, to_ja_input)]
to_zh_input = [prefix + ": " + input_text for prefix, input_text in zip(to_zh_prefix, to_zh_input)]
to_zh_input[:5]

['cejc apology res: query: はーい。 context: はい。ごめん。二合分のね。うん。',
 'cejc apology res: query: はーい。 context: すいません。あー。',
 'cejc apology res: query: だいじょぶです。 context: でも先生はもう。あー。ごめんなさい。な。',
 'cejc apology res: query: こうこうじゃん。 context: うん?。ちょっとごめんね。こう。こっち側ジャイアンツなんだ。',
 'cejc apology res: query: こっち?。＃＃。 context: 生ビール失礼いたします。すいません。わさびが効いてる。']

In [11]:
# Predict
to_ja_preds = model.predict(to_ja_input)
# to_ja_bleu = sacrebleu.corpus_bleu(to_ja_preds, to_ja_truth)
# print("--------------------------")
# print("to_ja_bleu: ", to_ja_bleu.score)

to_zh_preds = model.predict(to_zh_input)
# to_zh_bleu = sacrebleu.corpus_bleu(to_zh_preds, to_zh_truth)
# print("--------------------------")
# print("to_zh_bleu: ", to_zh_bleu.score)

In [12]:
r_ja_df = pd.DataFrame([to_ja_preds,to_ja_truth],index=[f'{ver_name}', 'truth'])
r_ja_df.T.to_csv(save_dir+'ja_preds_truth.csv',encoding='utf_8_sig')

r_zh_df = pd.DataFrame([to_zh_preds,to_zh_truth],index=[f'{ver_name}', 'truth'])
r_zh_df.T.to_csv(save_dir+'zh_preds_truth.csv',encoding='utf_8_sig')